*Defines a class for containing the thin wall topographic data and grid data*

In [71]:
import numpy
import netCDF4

In [177]:
class Mesh:
    """Describes 2D meshes.
    
    Meshes have shape=(nj,ni) cells with (nj+1,ni+1) vertices with coordinates (x,y).
    
    When constructing, either provide 1d or 2d coordinates (x,y), or assume a
    uniform spherical grid with 'shape' cells covering the whole sphere with
    longitudes starting at x0.
    
    Attributes:
    
    shape - (nj,ni)
    ni    - number of cells in x-direction (last)
    nj    - number of cells in y-direction (first)
    x     - longitude of mesh (cell corners), shape (nj+1,ni=1)
    y     - latitude of mesh (cell corners), shape (nj+1,ni=1)
    area  - area of cells, shape (nj,ni)
    """

    def __init__(self, shape=None, x=None, y=None, area=None, x0=-180.):
        """Constructor for Mesh:
        shape - shape of cell array, (nj,ni)
        ni    - number of cells in x-direction (last index)
        nj    - number of cells in y-direction (first index)
        x     - longitude of mesh (cell corners) (1d or 2d)
        y     - latitude of mesh (cell corners) (1d or 2d)
        area  - area of cells (2d)
        x0    - used when generating a spherical grid in absence of (x,y)
        """
        if (shape is None) and (x is None) and (y is None): raise Exception('Either shape must be specified or both x and y')
        if (x is None) and (y is not None): raise Exception('Either shape must be specified or both x and y')
        if (x is not None) and (y is None): raise Exception('Either shape must be specified or both x and y')
        # Determine shape
        if shape is not None:
            (nj,ni) = shape
        else: # Determine shape from x and y
            if (x is None) or (y is None): raise Exception('Either shape must be specified or both x and y')
            if len(x.shape)==1: ni = x.shape[0]-1
            elif len(x.shape)==2: ni = x.shape[1]-1
            else: raise Exception('x must be 1D or 2D.')
            if len(y.shape)==1 or len(y.shape)==2: nj = y.shape[0]-1
            else: raise Exception('y must be 1D or 2D.')
        self.ni = ni
        self.nj = nj
        self.shape = (nj,ni)
        # Check shape of arrays and construct 2d coordinates
        if x is not None and y is not None:
            if len(x.shape)==1:
                if len(y.shape)>1: raise Exception('x and y must either be both 1d or both 2d')
                if x.shape[0] != ni+1: raise Exception('x has the wrong length')
            if len(y.shape)==1:
                if len(x.shape)>1: raise Exception('x and y must either be both 1d or both 2d')
                if y.shape[0] != nj+1: raise Exception('y has the wrong length')
            if len(x.shape)==2 and len(y.shape)==2:
                if x.shape != y.shape: raise Exception('x and y are 2d and must be the same size')
                if x.shape != (nj+1,ni+1): raise Exception('x has the wrong size')
                self.x = x
                self.y = y
            else:
                self.x, self.y = numpy.meshgrid(x,y)
        else: # Construct coordinates
            y1d = numpy.linspace(-90.,90.,nj+1)
            x1d = numpy.linspace(x0,x0+360.,ni+1)
            self.x, self.y = numpy.meshgrid(x1d,y1d)
        if area is not None:
            if area.shape != (nj,ni): raise Exception('area has the wrong shape or size')
            self.area = area
        else:
            self.area = None
    def __repr__(self):
        return '<Mesh ni:%i nj:%i shape:(%i,%i)>'%(self.ni,self.nj,self.shape[0],self.shape[1])
    def dump(self):
        print(self)
        print('x =',self.x)
        print('y =',self.y)
        print('area =',self.area)
    def rhumb_length(p1, p2):
        """Distance between two points, p1 and p2, following a constant bearing.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        deg2rad = numpy.pi / 180.
        phi1 = deg2rad * p1[0]
        phi2 = deg2rad * p2[0]
        dphi = phi2 - phi1
        dlambda2 = ( deg2rad * ( p2[1] - p1[1] ) )**2
        dpsi = numpy.log( numpy.tan( 0.25 * numpy.pi + 0.5*phi2 ) / numpy.tan( 0.25*numpy.pi + 0.5*phi1 ) )
        if numpy.abs(dphi)==0: q2 = numpy.cos( 0.5*( phi1 + phi2 ) )**2
        else: q2 = ( dphi / dpsi )**2
        return numpy.sqrt( dphi**2 + q2 * dlambda2 )
    def angle_p1p2(p1, p2):
        """Angle at center of sphere between two points on the surface of the sphere.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        deg2rad = numpy.pi / 180.
        phi1 = deg2rad * p1[0]
        phi2 = deg2rad * p2[0]
        dphi_2 = 0.5 * ( phi2 - phi1 )
        dlambda_2 = 0.5 * deg2rad * ( p2[1] - p1[1] )
        a = numpy.sin( dphi_2 )**2 + numpy.cos( phi1 ) * numpy.cos( phi2 ) * ( numpy.sin( dlambda_2 )**2 )
        c = 2. * numpy.arctan2( numpy.sqrt(a), numpy.sqrt( 1. - a ) )
        return c
    def great_arc_length(p1, p2, R=1.):
        """Shortest distance between two points, p1 and p2.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        return R*Mesh.angle_p1p2(p1,p2)
    def angle_triangle(pA, pB, pC):
        """Angle ABC (angle between AC at B)
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        deg2rad = numpy.pi/180.
        a = Mesh.angle_p1p2(pA, pB)
        b = Mesh.angle_p1p2(pB, pC)
        c = Mesh.angle_p1p2(pC, pA)
        cos_c = numpy.cos(c)
        cos_b = numpy.cos(b)
        sin_b = numpy.sin(b)
        cos_a = numpy.cos(a)
        sin_a = numpy.sin(a)
        return numpy.arccos( ( cos_c - cos_b*cos_a) / (sin_a * sin_b) )
    def area_spherical_triangle(p1, p2, p3):
        """Area of a spherical triangle defined by great arcs between three positions on
        the surface of a sphere.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        A = Mesh.angle_triangle(p1, p2, p3)
        B = Mesh.angle_triangle(p2, p3, p1)
        C = Mesh.angle_triangle(p3, p1, p2)
        return (A+B+C-numpy.pi)
    def area_ggr_triangle(pA, pB, pC, n=5):
        """Area of a triangle ABC where AB and BC are great arcs and AC is a rhumb
        line (or straight line in geographic coordinates).
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        area = 0.*pB[0]
        for j in range(n):
            sL = float(j)/float(n)
            pL = ((1.-sL)*pA[0] + sL*pC[0], (1.-sL)*pA[1] + sL*pC[1])
            sR = float(j+1)/float(n)
            pR = ((1.-sR)*pA[0] + sR*pC[0], (1.-sR)*pA[1] + sR*pC[1])
            area += Mesh.area_spherical_triangle(pR,pB,pL)
        return area
    def __area_grid_cell__(p0, p1, p2, p3):
        p02 = (0.5*(p0[0] + p2[0]), 0.5*(p0[1] + p2[1]))
        p13 = (0.5*(p1[0] + p3[0]), 0.5*(p1[1] + p3[1]))
        p0123 = (0.5*(p02[0] + p13[0]), 0.5*(p02[1] + p13[1]))
        area = Mesh.area_ggr_triangle(p0, p01234, p1)
        area += Mesh.area_ggr_triangle(p1, p01234, p2)
        area += Mesh.area_ggr_triangle(p2, p01234, p3)
        area += Mesh.area_ggr_triangle(p3, p01234, p0)
        return area
    def set_area_to_spherical_grid(self, R=1.):
        """Sets the cell areas to those of a simple spherical grid."""
        deg2rad = numpy.pi / 180.
        lats = 0.5 * ( self.y[:,:-1] + self.y[:,1:] ) * deg2rad # i-average
        lons = 0.5 * ( self.x[:-1,:] + self.x[1:,:] ) # j-average
        dlon = ( lons[:,1:] - lons[:,:-1] ) *deg2rad
        self.area = dlon * ( numpy.sin(lats[1:,:]) - numpy.sin(lats[:-1,:]) ) * (R*R)
    def refine(self):
        """Returns new Mesh instance with twice the resolution"""
        x = numpy.zeros( (2*self.nj+1, 2*self.ni+1) )
        y = numpy.zeros( (2*self.nj+1, 2*self.ni+1) )
        #area = numpy.zeros( (2*self.nj, 2*self.ni) )
        x[::2,::2] = self.x
        x[::2,1::2] = 0.5 * ( self.x[:,:-1] + self.x[:,1:] )
        x[1::2,::2] = 0.5 * ( self.x[:-1,:] + self.x[1:,:] )
        x[1::2,1::2] = 0.25 * ( ( self.x[:-1,:-1] + self.x[1:,1:] ) + ( self.x[:-1,1:] + self.x[1:,:-1] ) )
        y[::2,::2] = self.y
        y[::2,1::2] = 0.5 * ( self.y[:,:-1] + self.y[:,1:] )
        y[1::2,::2] = 0.5 * ( self.y[:-1,:] + self.y[1:,:] )
        y[1::2,1::2] = 0.25 * ( ( self.y[:-1,:-1] + self.y[1:,1:] ) + ( self.y[:-1,1:] + self.y[1:,:-1] ) )
        return Mesh(x=x, y=y)
M = Mesh(shape=(2,4))
print( Mesh.rhumb_length((0,0),(90,0))/numpy.pi, 0.5)
print( Mesh.rhumb_length((0,0),(0,90))/numpy.pi, 0.5)
print( Mesh.great_arc_length((0,0),(90,0))/numpy.pi, 0.5)
print( Mesh.great_arc_length((0,0),(0,90))/numpy.pi, 0.5)
print( Mesh.angle_p1p2((0,0),(90,0))/numpy.pi*180., 90.)
print( Mesh.angle_triangle( (1,0), (0,0), (0,90) )/numpy.pi*180., 90.)
print( Mesh.area_spherical_triangle( (0,0), (90,0), (0,90) )/numpy.pi*8, 4.)
print( Mesh.area_ggr_triangle( (0,0), (90,0), (0,90), n=2 )/numpy.pi*8, 4. )
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=1 )/numpy.pi*8, 4., 1)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=2 )/numpy.pi*8, 4., 2)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=5 )/numpy.pi*8, 4., 5)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=20 )/numpy.pi*8, 4., 20)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=200 )/numpy.pi*8, 4., 200)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=2000 )/numpy.pi*8, 4., '2k')
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=5000 )/numpy.pi*8, 4., '5k')
#print( Mesh.area_spherical_triangle( (45,0), (0,0), (45,45) )/numpy.pi*8 )
#print( Mesh.area_rhumb_triangle( (45,0), (0,0), (45,45) )/numpy.pi*8 )
M.set_area_to_spherical_grid()
M.dump()
MM = M.refine()
MM.set_area_to_spherical_grid()
MM.dump()

0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
90.0 90.0
90.0 90.0
4.0 4.0
4.0 4.0
0.000387870637108 4.0 1
0.000387848484977 4.0 2
0.000387842283732 4.0 5
0.000387841220132 4.0 20
0.000387843909749 4.0 200
0.000387796099978 4.0 2k
0.000387084130943 4.0 5k
<Mesh ni:4 nj:2 shape:(2,4)>
x = [[-180.  -90.    0.   90.  180.]
 [-180.  -90.    0.   90.  180.]
 [-180.  -90.    0.   90.  180.]]
y = [[-90. -90. -90. -90. -90.]
 [  0.   0.   0.   0.   0.]
 [ 90.  90.  90.  90.  90.]]
area = [[ 1.57079633  1.57079633  1.57079633  1.57079633]
 [ 1.57079633  1.57079633  1.57079633  1.57079633]]
<Mesh ni:8 nj:4 shape:(4,8)>
x = [[-180. -135.  -90.  -45.    0.   45.   90.  135.  180.]
 [-180. -135.  -90.  -45.    0.   45.   90.  135.  180.]
 [-180. -135.  -90.  -45.    0.   45.   90.  135.  180.]
 [-180. -135.  -90.  -45.    0.   45.   90.  135.  180.]
 [-180. -135.  -90.  -45.    0.   45.   90.  135.  180.]]
y = [[-90. -90. -90. -90. -90. -90. -90. -90. -90.]
 [-45. -45. -45. -45. -45. -45. -45. -45. -45.]
 [  0.  

In [193]:
class NodeData:
    """2D node-registered data
    
    data - data on nodes, shape (nj,ni)
    x - longitude of node, shape (ni)
    y - latitude of node, shape (nj)
    """
    def __init__(self, data, x, y):
        self.data = data
        self.shape = self.data.shape
        self.ni = self.shape[1]
        self.nj = self.shape[0]
        self.x = x
        self.y = y
    @classmethod
    def init_from_file(cls, filename, variable_name):
        """Initialize from a netcdf file "filename" using variable "variable_name"."""
        rg = netCDF4.Dataset(filename)
        data = rg.variables[variable_name]
        def find_var_with_dim(rg,var,dim):
            for v in rg.variables:
                if len(rg.variables[v].shape)==1 and dim in rg.variables[v].dimensions:
                    return rg.variables[v]
            raise Exception('No coordinate data!')
        dims = data.dimensions
        x = find_var_with_dim(rg,data,dims[1])
        y = find_var_with_dim(rg,data,dims[0])
        return cls(data, x, y)
    def subset(self, x0, x1, y0, y1):
        """Returns new NodeData instance subset of NodeData."""
        # Assume -90<y0,y1<90
        j0 = numpy.argmin( numpy.abs(self.y[:]-y0) )
        j1 = numpy.argmin( numpy.abs(self.y[:]-y1) )
        # Assume regular x spacing (this allows for wrapping)
        dx = 360./self.ni
        i0 = int( ( x0 - self.x[0] ) / dx ) # Rounds down
        i1 = int( ( x1 - self.x[0] ) / dx +.99999) # Rounds up
        print((self.x[1]-self.x[0]),360./self.ni)
        print(i0,i1,j0,j1)
        # Todo: Needs logic for wrapping...
        return NodeData(self.data[j0:j1+1,i0:i1+1], self.x[i0:i1+1], self.y[j0:j1+1])
D = NodeData.init_from_file('ETOPO5.nc','elevation')
print(D.shape)
DD = D.subset(-100,-60,5,35)
print(DD.shape)

(2161, 4320)
0.0833333 0.08333333333333333
-1200 -719 1140 1500
(361, 482)


In [197]:
class ThinWall(Mesh):
    """Container for thin wall topographic data and mesh/grid data

    zc_mean - mean elevation of cell, shape (nj,ni)
    zu_mean - mean elevation of western edge of cell, shape (nj,ni+1)
    zv_mean - mean elevation of southern edge of cell, shape (nj+1,nj)
    """

    def __init__(self, *args, **kwargs):
        Mesh.__init__(self, *args, **kwargs)
        self.shapeu = (self.nj, self.ni+1)
        self.shapev = (self.nj+1, self.ni)
        self.zc_mean = None
        self.zu_mean = None
        self.zv_mean = None
    def dump(self):
        super().dump()
        print('zc_mean =',self.zc_mean)
        print('zu_mean =',self.zu_mean)
        print('zv_mean =',self.zv_mean)
    def refine(self):
        """Returns new ThinWall instance with twice the resolution"""
        M = super().refine()
        M.__class__ = ThinWall # Is this acceptable?
        M.zc_mean = None
        M.zu_mean = None
        M.zv_mean = None
        return M
    def set_cell_mean(self, data):
        self.zc_mean = 1.*data
    def set_edge_mean(self, datau, datav):
        self.zu_mean = 1.*datau
        self.zv_mean = 1.*datav
    def set_edge_mean_to_step(self):
        self.zu_mean = numpy.zeros(self.shapeu)
        self.zu_mean[:,1:-1] = numpy.maximum( self.zc_mean[:,:-1], self.zc_mean[:,1:] )
        self.zu_mean[:,0] = numpy.maximum( self.zc_mean[:,0], self.zc_mean[:,-1] )
        self.zu_mean[:,-1] = numpy.maximum( self.zc_mean[:,0], self.zc_mean[:,-1] )
        self.zv_mean = numpy.zeros(self.shapev)
        self.zv_mean[1:-1,:] = numpy.maximum( self.zc_mean[:-1,:], self.zc_mean[1:,:] )
        self.zv_mean[0,:] = numpy.maximum( self.zc_mean[0,:], self.zc_mean[-1,:] )
        self.zv_mean[-1,:] = numpy.maximum( self.zc_mean[0,:], self.zc_mean[-1,:] )
    def min_dxdy(self):
        return numpy.min( self.y[1:,:] - self.y[:-1,:] )
T = ThinWall(x=numpy.linspace(-100,-60,41), y=numpy.linspace(5,35,31))
T.set_area_to_spherical_grid()
#T.set_cell_mean( numpy.random.rand(2,4) )
#T.set_edge_mean_to_step()
T.dump()
#TT = T.refine()
#TT.dump()

<Mesh ni:40 nj:30 shape:(30,40)>
x = [[-100.  -99.  -98. ...,  -62.  -61.  -60.]
 [-100.  -99.  -98. ...,  -62.  -61.  -60.]
 [-100.  -99.  -98. ...,  -62.  -61.  -60.]
 ..., 
 [-100.  -99.  -98. ...,  -62.  -61.  -60.]
 [-100.  -99.  -98. ...,  -62.  -61.  -60.]
 [-100.  -99.  -98. ...,  -62.  -61.  -60.]]
y = [[  5.   5.   5. ...,   5.   5.   5.]
 [  6.   6.   6. ...,   6.   6.   6.]
 [  7.   7.   7. ...,   7.   7.   7.]
 ..., 
 [ 33.  33.  33. ...,  33.  33.  33.]
 [ 34.  34.  34. ...,  34.  34.  34.]
 [ 35.  35.  35. ...,  35.  35.  35.]]
area = [[ 0.00030321  0.00030321  0.00030321 ...,  0.00030321  0.00030321
   0.00030321]
 [ 0.00030266  0.00030266  0.00030266 ...,  0.00030266  0.00030266
   0.00030266]
 [ 0.00030201  0.00030201  0.00030201 ...,  0.00030201  0.00030201
   0.00030201]
 ..., 
 [ 0.00025691  0.00025691  0.00025691 ...,  0.00025691  0.00025691
   0.00025691]
 [ 0.00025401  0.00025401  0.00025401 ...,  0.00025401  0.00025401
   0.00025401]
 [ 0.00025104  0.00025104  

In [198]:
def recurse_until_res(T, res=1./12.):
    print(T.shape)
    if T.shape[0]>9000: raise Exception('Oops')
    if T.min_dxdy() > res: return [T]+recurse_until_res(T.refine(), res=res)
    return [T]
ttt = recurse_until_res(T)
ttt[-1].x

(30, 40)
(60, 80)
(120, 160)
(240, 320)
(480, 640)


array([[-100.    ,  -99.9375,  -99.875 , ...,  -60.125 ,  -60.0625,  -60.    ],
       [-100.    ,  -99.9375,  -99.875 , ...,  -60.125 ,  -60.0625,  -60.    ],
       [-100.    ,  -99.9375,  -99.875 , ...,  -60.125 ,  -60.0625,  -60.    ],
       ..., 
       [-100.    ,  -99.9375,  -99.875 , ...,  -60.125 ,  -60.0625,  -60.    ],
       [-100.    ,  -99.9375,  -99.875 , ...,  -60.125 ,  -60.0625,  -60.    ],
       [-100.    ,  -99.9375,  -99.875 , ...,  -60.125 ,  -60.0625,  -60.    ]])

In [201]:
D